In [ ]:
import pandas as pd
import os
from alchemlyb.parsing import amber
from alchemlyb.estimators import TI,MBAR
from alchemlyb.postprocessors.units import to_kcalmol, to_kJmol, to_kT


In [ ]:
top = os.getcwd()
result_dir = ['complex_1_20_200_200', 'complex_7_20_200_200', 'complex_8_20_200_200']
os.chdir('./no_restraint_mix')
file_list = [0.0, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
dHdl = pd.concat([amber.extract_dHdl(f'{top}/{dir}/vdw_crg_one_step/{filename}/ti001.out', T=300) for dir in result_dir for filename in file_list])
u_nk_rm_rest = pd.concat([amber.extract_u_nk(f'{top}/{dir}/vdw_crg_one_step/{filename}/ti001.out', T=300) for dir in result_dir for filename in file_list])

In [ ]:
u_nk_rm_rest.shape

In [ ]:
u_nk_rm_rest

In [ ]:
ti_estimator = TI().fit(dHdl)
mbar_estimator = MBAR().fit(u_nk_rm_rest)

In [ ]:
print('MBAR free energy output is\n', mbar_estimator.delta_f_)

print('energy unit is',  mbar_estimator.delta_f_.attrs['energy_unit'])
print('free energy difference is', mbar_estimator.delta_f_.loc[0.00, 1.00])

delta_f_kcalmol = to_kcalmol(mbar_estimator.delta_f_)
print('energy unit is', delta_f_kcalmol.attrs['energy_unit'])
print('free energy difference is now', delta_f_kcalmol.loc[0.00, 1.00])

In [ ]:
delta_f = ti_estimator.delta_f_.iloc[0, -1]
d_delta_f = ti_estimator.d_delta_f_.iloc[0, -1]

In [ ]:
print("TI:")
print("Free energy difference (TI)", delta_f, "kT")
print("Free energy difference (TI) error", d_delta_f, "kT")
print("Free energy difference (MBAR)", mbar_estimator.delta_f_.loc[0.0, 1.0], 'kT')
print("Free energy difference (MBAR) error", mbar_estimator.d_delta_f_.loc[0.0, 1.0], 'kT')


In [ ]:
from alchemlyb.visualisation import plot_ti_dhdl,plot_mbar_overlap_matrix
ax = plot_ti_dhdl([ti_estimator], labels=['dHdl'], colors=['r'],units='kcal/mol')
ax.figure.savefig('onestep_dhdl_TI.pdf')

In [ ]:
ax = plot_mbar_overlap_matrix(mbar_estimator.overlap_matrix)
ax.figure.savefig('onestep_MBAR.pdf', bbox_inches='tight', pad_inches=0.0)

In [ ]:
import matplotlib.pyplot as plt
from alchemlyb.visualisation.dF_state import plot_dF_state

In [ ]:
estimators = [(ti_estimator), (mbar_estimator)]

fig = plot_dF_state(estimators, orientation='portrait',units='kcal/mol')
fig.savefig('dF_state.pdf', bbox_inches='tight')

file_path = os.getcwd() + '/resust.txt'
with open(file_path,'w') as f:
    f.write(f'Free energy difference(TI): {delta_f * 0.596:.3f} ± {d_delta_f * 0.596:.3f} Kcal/mol\n') 
    f.write(f"Free energy difference (MBAR): {(mbar_estimator.delta_f_.loc[0.0, 1.0] * 0.596):.3f} ± {(mbar_estimator.d_delta_f_.loc[0.0, 1.0] * 0.596):.3f} Kcal/mol\n")
    for i in range(len(file_list)-1):
        f.writelines(f'{mbar_estimator.delta_f_.loc[file_list[i],file_list[i+1]] * 0.596:.3f}\n')

f.close()